In [11]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain import LLMChain, PromptTemplate

from langchain_openai import ChatOpenAI

In [3]:
load_dotenv()
assert os.getenv("OPENAI_API_KEY"), "Defina OPENAI_API_KEY em .env"

In [5]:
PATH = "cleaned_list.xlsx"  # ou "cleaned_list.xlsx" se estiver na mesma pasta
df = pd.read_excel(PATH, dtype=str)

# Limitando aos 30 primeiros itens
df = df.head(30)
print(f"Linhas carregadas: {len(df)}")
print("Limitado aos 30 primeiros itens")

Linhas carregadas: 1000


In [7]:
prompt = PromptTemplate(
    input_variables=["descricao"],
    template=(
        "Dado o nome de um produto, responda com o NOME DA MARCA associada.\n"
        "Se não houver marca, responda exatamente: Nenhuma.\n\n"
        "Exemplos:\n"
        "- Neutrogena Sun Fresh FPS 70 200 ML → Neutrogena\n"
        "- Nescau Achocolatado em pó 400g → Nescau\n"
        "- Papel A4 200x300 UND → Nenhuma\n\n"
        "Produto: {descricao}\n"
        "Marca:"
    ),
)

In [12]:
llm = ChatOpenAI(model="gpt-4", temperature=0)
chain = LLMChain(llm=llm, prompt=prompt)

In [13]:
def classify_brands(df: pd.DataFrame, desc_col: str) -> pd.Series:
    """
    Para cada descrição em df[desc_col], executa o LLMChain e retorna uma Series de marcas.
    """
    marcas = []
    for i, txt in enumerate(df[desc_col].fillna("").astype(str)):
        if not txt.strip():
            marcas.append("Nenhuma")
        else:
            resp = chain.run(descricao=txt).strip()
            marcas.append(resp)
        if (i + 1) % 100 == 0:
            print(f"Processados {i+1}/{len(df)}")
    return pd.Series(marcas, index=df.index)

In [21]:
COL_DESC = "Produto/Serviço"
df = df.head(300).copy()
df["Marca"] = classify_brands(df, COL_DESC)
df_com_marca = df[df["Marca"].str.lower() != "nenhuma"]
print(f"Encontrados {len(df_com_marca)} produtos com marca.")

# Extrair lista de todas as marcas únicas (excluindo "nenhuma")
lista_marcas = df_com_marca["Marca"].unique().tolist()
print(f"Total de marcas únicas encontradas: {len(lista_marcas)}")
print("Marcas encontradas:")
for marca in sorted(lista_marcas):
    print(f"- {marca}")

# Salvar o DataFrame com a nova coluna "Marca"
# Opção 1: Salvar como CSV
df.to_csv("produtos_com_marcas.csv", index=False, encoding="utf-8")
print("\nDataFrame salvo como 'produtos_com_marcas.csv'")

# Opção 2: Salvar como Excel (descomente se preferir)
# df.to_excel("produtos_com_marcas.xlsx", index=False)
# print("DataFrame salvo como 'produtos_com_marcas.xlsx'")

# Opção 3: Salvar como Parquet (mais eficiente para grandes datasets)
# df.to_parquet("produtos_com_marcas.parquet", index=False)
# print("DataFrame salvo como 'produtos_com_marcas.parquet'")

# Mostrar algumas estatísticas
print(f"\nResumo:")
print(f"Total de produtos: {len(df)}")
print(f"Produtos com marca identificada: {len(df_com_marca)}")
print(f"Produtos sem marca: {len(df) - len(df_com_marca)}")
print(f"Marcas únicas: {len(lista_marcas)}")

# Visualizar as primeiras linhas do DataFrame com a nova coluna
print(f"\nPrimeiras 5 linhas do DataFrame com a coluna Marca:")
print(df[["Produto/Serviço", "Marca"]].head())

Encontrados 6 produtos com marca.
Total de marcas únicas encontradas: 2
Marcas encontradas:
- Nenhuma.
- Neutrogena

DataFrame salvo como 'produtos_com_marcas.csv'

Resumo:
Total de produtos: 10
Produtos com marca identificada: 6
Produtos sem marca: 4
Marcas únicas: 2

Primeiras 5 linhas do DataFrame com a coluna Marca:
                           Produto/Serviço       Marca
0             NEVRALGEX DIP 1 G C/ 10 COMP     Nenhuma
1                NEVRIX 5000 MCG 3 AMPOLAS    Nenhuma.
2       NEUTROGENA SUN FRESH FPS 90 200 ML  Neutrogena
3  NEUTROGENA NORWEGIAN HIDRAT CORP 200 ML  Neutrogena
4       NEUTROGENA SUN FRESH FPS 70 200 ML  Neutrogena
